# Домашнее задание 2. Классификация изображений.

В этом задании потребуется обучить классификатор изображений. Будем работать с датасетом, название которого раскрывать не будем. Можете посмотреть самостоятельно на картинки, которые в есть датасете. В нём 200 классов и около 5 тысяч картинок на каждый класс. Классы пронумерованы, как нетрудно догадаться, от 0 до 199. Скачать датасет можно вот [тут](https://yadi.sk/d/BNR41Vu3y0c7qA).

Структура датасета простая -- есть директории train/ и val/, в которых лежат обучающие и валидационные данные. В train/ и val/ лежат директориии, соответствующие классам изображений, в которых лежат, собственно, сами изображения.
 
__Задание__. Необходимо выполнить любое из двух заданий

1) Добейтесь accuracy **на валидации не менее 0.44**. В этом задании **запрещено** пользоваться предобученными моделями и ресайзом картинок. 

2) Добейтесь accuracy **на валидации не менее 0.84**. В этом задании делать ресайз и использовать претрейн можно. 

Напишите краткий отчёт о проделанных экспериментах. Что сработало и что не сработало? Почему вы решили, сделать так, а не иначе? Обязательно указывайте ссылки на чужой код, если вы его используете. Обязательно ссылайтесь на статьи / блогпосты / вопросы на stackoverflow / видосы от ютуберов-машинлернеров / курсы / подсказки от Дяди Васи и прочие дополнительные материалы, если вы их используете. 

Ваш код обязательно должен проходить все `assert`'ы ниже.

Необходимо написать функции `train_one_epoch`, `train` и `predict` по шаблонам ниже (во многом повторяют примеры с семинаров).Обратите особое внимание на функцию `predict`: она должна возвращать список лоссов по всем объектам даталоадера, список предсказанных классов для каждого объекта из даталоалера и список настоящих классов для каждого объекта в даталоадере (и именно в таком порядке).

__Использовать внешние данные для обучения строго запрещено в обоих заданиях. Также запрещено обучаться на валидационной выборке__.


__Критерии оценки__: Оценка вычисляется по простой формуле: `min(10, 10 * Ваша accuracy / 0.44)` для первого задания и `min(10, 10 * (Ваша accuracy - 0.5) / 0.34)` для второго. Оценка округляется до десятых по арифметическим правилам. Если вы выполнили оба задания, то берется максимум из двух оценок.

__Бонус__. Вы получаете 5 бонусных баллов если справляетесь с обоими заданиями на 10 баллов (итого 15 баллов). В противном случае выставляется максимальная из двух оценок и ваш бонус равен нулю.

__Советы и указания__:
 - Наверняка вам потребуется много гуглить о классификации и о том, как заставить её работать. Это нормально, все гуглят. Но не забывайте, что нужно быть готовым за скатанный код отвечать :)
 - Используйте аугментации. Для этого пользуйтесь модулем `torchvision.transforms` или библиотекой [albumentations](https://github.com/albumentations-team/albumentations)
 - Можно обучать с нуля или файнтюнить (в зависимости от задания) модели из `torchvision`.
 - Рекомендуем написать вам сначала класс-датасет (или воспользоваться классом `ImageFolder`), который возвращает картинки и соответствующие им классы, а затем функции для трейна по шаблонам ниже. Однако делать это мы не заставляем. Если вам так неудобно, то можете писать код в удобном стиле. Однако учтите, что чрезмерное изменение нижеперечисленных шаблонов увеличит количество вопросов к вашему коду и повысит вероятность вызова на защиту :)
 - Валидируйте. Трекайте ошибки как можно раньше, чтобы не тратить время впустую.
 - Чтобы быстро отладить код, пробуйте обучаться на маленькой части датасета (скажем, 5-10 картинок просто чтобы убедиться что код запускается). Когда вы поняли, что смогли всё отдебажить, переходите обучению по всему датасету
 - На каждый запуск делайте ровно одно изменение в модели/аугментации/оптимайзере, чтобы понять, что и как влияет на результат.
 - Фиксируйте random seed.
 - Начинайте с простых моделей и постепенно переходите к сложным. Обучение лёгких моделей экономит много времени.
 - Ставьте расписание на learning rate. Уменьшайте его, когда лосс на валидации перестаёт убывать.
 - Советуем использовать GPU. Если у вас его нет, используйте google colab. Если вам неудобно его использовать на постоянной основе, напишите и отладьте весь код локально на CPU, а затем запустите уже написанный ноутбук в колабе. Авторское решение задания достигает требуемой точности в колабе за 15 минут обучения.
 
Good luck & have fun! :)

In [4]:
!pip install torchvision

     |████████████████████████████████| 1.2 MB 961 kB/s            


In [68]:
import numpy as np
import torch
import torchvision
from tqdm.notebook import tqdm
from torch import nn
from torch.nn import functional as F
from sklearn.metrics import accuracy_score

# You may add any imports you need
from torchvision.transforms import Compose, Normalize, Resize, ToTensor
import matplotlib.pyplot as plt

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

### Подготовка данных

In [ ]:
# class MyDataset(torch.utils.data.Dataset):
#     def __init__(self, data_dir, transform):
#         # YOUR CODE
#         pass
    
#     def __getitem__(self, idx):
#         # YOUR CODE
#         pass
    
#     def __len__(self):
#         # YOUR CODE
#         pass

In [15]:
train_transform = Compose(
    [ToTensor(),
     Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
)
val_transform = ToTensor()
# YOU CAN DEFINE AUGMENTATIONS HERE

train_dataset = torchvision.datasets.ImageFolder(root="./dataset/dataset/train", transform=train_transform)
val_dataset = torchvision.datasets.ImageFolder(root="./dataset/dataset/val", transform=val_transform)
# REPLACE ./dataset/dataset WITH THE FOLDER WHERE YOU DOWNLOADED AND UNZIPPED THE DATASET
# OR USE torchvision.datasets.ImageFolder INSTEAD OF MyDataset

train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=256, shuffle=True)
val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=256, shuffle=True)

In [16]:
# Just very simple sanity checks
assert isinstance(train_dataset[0], tuple)
assert len(train_dataset[0]) == 2
assert isinstance(train_dataset[1][1], int)
print("tests passed")

tests passed


### Вспомогательные функции, реализация модели

In [88]:
def train_one_epoch(model, train_dataloader, criterion, optimizer, device="cuda:0"):
    model.train()
    # YOUR CODE
    # TRAIN YOUR MODEL HERE
    # train_losses, train_acc = 0., 0., 0.
    for imgs, labels in tqdm(train_dataloader, desc=f"Training",  leave=False):
        # tensors go to device, where model is
        imgs, labels = imgs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        y_pred = model(imgs) # матрица уверенностей
        loss = criterion(y_pred, labels)
        loss.backward()
        optimizer.step()


def predict(model, val_dataloder, criterion, device="cuda:0"):
    model.eval()
    # YOUR CODE
    losses = []
    predicted_classes = []
    true_classes = []
    with torch.no_grad():
        for imgs, labels in tqdm(val_dataloader, leave=False):
            # tensors go to device, where model is
            imgs, labels = imgs.to(device), labels.to(device)
            pred = model(imgs)
            loss = criterion(pred, labels)

            losses.append(loss)
            predicted_classes += torch.argmax(pred, dim=-1)
            # predicted_classes.append(pred_classes)
            true_classes += labels
        
    # PREDICT FOR EVERY ELEMENT OF THE VAL DATALOADER AND RETURN CORRESPONDING LISTS
    return losses, predicted_classes, true_classes


def train(model, train_dataloader, val_dataloader, criterion, optimizer, device="cuda:0", n_epochs=10, 
          scheduler=None):
    model.to(device)
    train_loss_log, train_acc_log, val_loss_log, val_acc_log = [], [], [], []
    for epoch in range(n_epochs):
        print(f'Epoch: {epoch + 1}:')
        train(model, train_dataloader, criterion, optimizer, device)
        
        train_losses, train_preds, train_classes = predict(model, train_dataloader, criterion, device)
        train_acc = accuracy_sscore(train_preds, train_classes)
        print(f'Train accuracy: {train_acc}')
        
        val_losses, val_preds, val_classes = predict(model, val_dataloader, criterion, device)
        val_acc = accuracy_score(val_preds, val_classes)
        print(f'Validation accuracy: {val_acc}')
        
        train_loss_log.append(train_losses.mean())
        val_loss_log.append(val_losses.mean())
        
        train_acc_log.append(train_acc)
        val_acc_log.append(val_acc)
        
    return train_loss_log, train_acc_log, val_loss_log, val_acc_log
    # Train, evaluate, print accuracy, make a step of scheduler or whatever you want...

In [64]:
def show_image(dataset_item):
    plt.imshow(np.transpose(dataset_item[0].numpy(), (1, 2, 0)), cmap='Greys')
    plt.show()

### Обучение модели, запуски экспериментов

In [65]:
class ModelBaseline(nn.Module):
    def __init__(self):
        super(ModelBaseline, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, padding=1) # + relu
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=16, kernel_size=5, padding=2) # + relu
        # max pool 2x2, s=2 + flatten
        self.fc1 = nn.Linear(32 * 32 * 16, 128)
        self.fc2 = nn.Linear(128, 200)
        
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.flatten = nn.Flatten()

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = self.flatten(self.pool(x))
        x = self.fc1(x)
        x = self.fc2(x)
        return x

In [86]:
model = ModelBaseline()
optimizer = torch.optim.AdamW(model.parameters(), lr=0.0001, weight_decay=0.01)
criterion = nn.CrossEntropyLoss()
scheduler = None # LR SCHEDULE THAT YOU PROBABLY CHOOSE
n_epochs = 10
device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")

Простой тест на проверку правильности написанного кода

In [89]:
all_losses, predicted_labels, true_labels = predict(model, val_dataloader, criterion, device)
assert len(predicted_labels) == len(val_dataset)
accuracy = accuracy_score(predicted_labels, true_labels)
print("tests passed")

  0%|          | 0/40 [00:00<?, ?it/s]

tests passed


Запустить обучение можно в ячейке ниже.

In [ ]:
train(model, train_dataloader, val_dataloader, criterion, optimizer, device, n_epochs, scheduler)

### Проверка полученной accuracy

После всех экспериментов которые вы проделали, выберите лучшую из своих моделей, реализуйте и запустите функцию `evaluate`. Эта функция должна брать на вход модель и даталоадер с валидационными данными и возврашать accuracy, посчитанную на этом датасете.

In [ ]:
all_losses, predicted_labels, true_labels = predict(model, val_dataloader, criterion, device)
assert len(predicted_labels) == len(val_dataset)
accuracy = accuracy_score(true_labels, predicted_labels)
print("Оценка за это задание составит {} баллов".format(min(5, 5 * accuracy / 0.44)))

### Отчёт об экспериментах 

текст писать тут